<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [22]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [23]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [24]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [25]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [26]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [27]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [28]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [29]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [30]:
!rm -rf logs

rm: cannot remove 'logs/20230601-054403/train': Directory not empty
rm: cannot remove 'logs/20230601-054403/validation': Directory not empty


In [31]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 23s 868ms/step - loss: 0.6717 - accuracy: 0.5718 - val_loss: 0.6598 - val_accuracy: 0.5610
Epoch 2/10
26/26 [==============================] - 23s 904ms/step - loss: 0.6175 - accuracy: 0.7433 - val_loss: 0.5747 - val_accuracy: 0.8000
Epoch 3/10
26/26 [==============================] - 23s 885ms/step - loss: 0.5306 - accuracy: 0.7968 - val_loss: 0.5034 - val_accuracy: 0.8341
Epoch 4/10
26/26 [==============================] - 21s 792ms/step - loss: 0.4601 - accuracy: 0.8078 - val_loss: 0.4652 - val_accuracy: 0.7463
Epoch 5/10
26/26 [==============================] - 21s 797ms/step - loss: 0.3730 - accuracy: 0.8613 - val_loss: 0.3340 - val_accuracy: 0.8244
Epoch 6/10
26/26 [==============================] - 20s 788ms/step - loss: 0.3164 - accuracy: 0.8966 - val_loss: 0.3833 - val_accuracy: 0.8585
Epoch 7/10
26/26 [==============================] - 20s 773ms/step - loss: 0.2500 - accuracy: 0.9197 - val_loss: 0.2508 - val_accuracy: 0.9317

In [32]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 1327), started 0:42:09 ago. (Use '!kill 1327' to kill it.)

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [33]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.68.h5
26/26 - 32s - loss: 0.6843 - accuracy: 0.5341 - val_loss: 0.6810 - val_accuracy: 0.4976
Epoch 2/5

Epoch 00003: saving model to weights.03-0.65.h5
26/26 - 20s - loss: 0.6306 - accuracy: 0.7141 - val_loss: 0.6451 - val_accuracy: 0.5415
Epoch 4/5

Epoch 00004: saving model to weights.04-0.53.h5
26/26 - 21s - loss: 0.5755 - accuracy: 0.7470 - val_loss: 0.5276 - val_accuracy: 0.7707
Epoch 5/5

Epoch 00005: saving model to weights.05-0.60.h5
26/26 - 20s - loss: 0.5273 - accuracy: 0.7798 - val_loss: 0.5984 - val_accuracy: 0.6244


In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])

In [35]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])


Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6287 - accuracy: 0.6606 - val_loss: 0.6360 - val_accuracy: 0.6000


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [36]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 21s - loss: 0.6668 - accuracy: 0.5706 - val_loss: 0.6976 - val_accuracy: 0.4439
Epoch 2/50
26/26 - 21s - loss: 0.5992 - accuracy: 0.7178 - val_loss: 0.5610 - val_accuracy: 0.7366
Epoch 3/50
26/26 - 21s - loss: 0.5245 - accuracy: 0.7482 - val_loss: 0.5208 - val_accuracy: 0.8098
Epoch 4/50
26/26 - 22s - loss: 0.4792 - accuracy: 0.7701 - val_loss: 0.4357 - val_accuracy: 0.7951
Epoch 5/50
26/26 - 20s - loss: 0.4201 - accuracy: 0.8163 - val_loss: 0.3956 - val_accuracy: 0.8537
Epoch 6/50
26/26 - 22s - loss: 0.3845 - accuracy: 0.8285 - val_loss: 0.3260 - val_accuracy: 0.8585
Epoch 7/50
26/26 - 21s - loss: 0.3391 - accuracy: 0.8686 - val_loss: 0.3015 - val_accuracy: 0.8878
Epoch 8/50
26/26 - 23s - loss: 0.3012 - accuracy: 0.8917 - val_loss: 0.3203 - val_accuracy: 0.8585
Epoch 9/50
26/26 - 20s - loss: 0.2416 - accuracy: 0.9209 - val_loss: 0.2452 - val_accuracy: 0.9073
Epoch 10/50
26/26 - 20s - loss: 0.1948 - accuracy: 0.9453 - val_loss: 0.1781 - val_accuracy: 0.9317
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [37]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 25s 968ms/step - loss: 0.6756 - accuracy: 0.6119 - val_loss: 0.6784 - val_accuracy: 0.4634
Epoch 2/5
26/26 [==============================] - 22s 856ms/step - loss: 0.6200 - accuracy: 0.6679 - val_loss: 0.6020 - val_accuracy: 0.6098
Epoch 3/5
26/26 [==============================] - 21s 793ms/step - loss: 0.5745 - accuracy: 0.7202 - val_loss: 0.7184 - val_accuracy: 0.4829
Epoch 4/5
26/26 [==============================] - 20s 776ms/step - loss: 0.5084 - accuracy: 0.7798 - val_loss: 0.4400 - val_accuracy: 0.8732
Epoch 5/5
26/26 [==============================] - 21s 791ms/step - loss: 0.4374 - accuracy: 0.8273 - val_loss: 0.3887 - val_accuracy: 0.9122


In [38]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.611922,0.675882,0.463415,0.678381
1,1,0.667883,0.619023,0.609756,0.602015
2,2,0.720195,0.575555,0.482927,0.718446
3,3,0.779805,0.508197,0.873171,0.440037
4,4,0.827251,0.436536,0.912195,0.388683


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [39]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 30s 1s/step - loss: 0.6809 - accuracy: 0.5474 - val_loss: 0.7071 - val_accuracy: 0.4341

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 24s 919ms/step - loss: 0.6605 - accuracy: 0.5742 - val_loss: 0.6647 - val_accuracy: 0.6732

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 21s 827ms/step - loss: 0.6508 - accuracy: 0.6691 - val_loss: 0.6641 - val_accuracy: 0.5805

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 21s 812ms/step - loss: 0.6450 - accuracy: 0.6618 - val_loss: 0.6631 - val_accuracy: 0.5659

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 786ms/step - loss: 0.6422

In [40]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4736), started 0:43:07 ago. (Use '!kill 4736' to kill it.)

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [41]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 21s 819ms/step - loss: 0.3281 - accuracy: 0.9051 - val_loss: 0.3139 - val_accuracy: 0.8976
Epoch 8/50
26/26 [==============================] - 21s 812ms/step - loss: 0.2978 - accuracy: 0.9197 - val_loss: 0.2882 - val_accuracy: 0.9220
Epoch 9/50
26/26 [==============================] - 21s 812ms/step - loss: 0.2845 - accuracy: 0.9282 - val_loss: 0.2712 - val_accuracy: 0.9268
Epoch 10/50
25/26 [===========================>..] - ETA: 0s - loss: 0.2707 - accuracy: 0.9300
Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.001.
26/26 [==============================] - 24s 915ms/step - loss: 0.2701 - accuracy: 0.9307 - val_loss: 0.2754 - val_accuracy: 0.9073
Epoch 11/50
26/26 [==============================] - 21s 804ms/step - loss: 0.2583 - accuracy: 0.9355 - val_loss: 0.2518 - val_accuracy: 0.9268
Epoch 12/50
25/26 [===========================>..] - ETA: 0s - loss: 0.2506 - accuracy: 0.9350
Epoch 00012: ReduceLROnPlateau reducing l

In [42]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4736), started 1:00:45 ago. (Use '!kill 4736' to kill it.)